# Basic SVD using 6% data! ( LB 0.01961)

### singular Value Decomposition(SVD) 概念說明
http://blog.csdn.net/u012162613/article/details/42214205

### 程式碼
### 前置處理
#### 引入套件

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.linalg import sqrtm
from copy import deepcopy

#### 資料讀取、資料和欄位挑選

In [2]:
usecols1 = ['fecha_dato', 'ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

train = pd.read_csv("/Users/micotai/Documents/kaggle/train_ver2.csv", usecols=usecols1)
train1 = train[train['fecha_dato']=="2016-05-28"].drop("fecha_dato", axis = 1)
train2 = train[train['fecha_dato']=="2016-04-28"].drop("fecha_dato", axis = 1)
#train3 = train[train['fecha_dato']=="2016-03-28"].drop("fecha_dato", axis = 1)

true = deepcopy(train1)

#### testing data讀取

In [3]:
test = pd.read_csv("/Users/micotai/Documents/kaggle/test_ver2.csv")
test = test[['ncodpers']]

print("datasets loaded")

datasets loaded


/Library/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### 將客戶編號視為key值

In [4]:
users = true['ncodpers'].tolist()
true.drop('ncodpers', axis=1, inplace=True)

items = true.columns.tolist()
u = {}
for i in range(len(users)):
    u[users[i]] = i

trueMat = np.array(true)
print("users dict formed")

users dict formed


In [5]:
def reorder(train):
    train.index = train['ncodpers'].tolist()
    train.drop('ncodpers', axis=1, inplace=True)
    train = train.reindex(users)
    return train

#train3 = reorder(train3)
train2 = reorder(train2)
train1 = reorder(train1)

### COMPUTING THE SVD

In [6]:
def svd(train, trueMat, k):
    utilMat = np.array(train)

    mask = np.isnan(utilMat)
    masked_arr=np.ma.masked_array(utilMat, mask)
    item_means=np.mean(masked_arr, axis=0)
    utilMat = masked_arr.filled(item_means)

    x = np.tile(item_means, (utilMat.shape[0],1))

    utilMat = utilMat - x

    print(utilMat)

    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]

    s_root=sqrtm(s)

    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)

    UsV = UsV + x

    UsV = np.ma.masked_where(trueMat==1,UsV).filled(fill_value=-999)

    #print(UsV)
    print("svd done")
    return UsV

full_matrices : bool, optional

If True (default), u and vh have the shapes (..., M, M) and (..., N, N), respectively. Otherwise, the shapes are (..., M, K) and (..., K, N), respectively, where K = min(M, N).

### PREDICTION

In [7]:
def max_items(UsV,x,j):
    out = []

    for xx in x:
        if UsV[j,xx]>0.001: # setting a threshold
            out.append(items[xx])

    return out

In [8]:
def recommend(test):

    UsV = (5*svd(train=train1,trueMat=trueMat,k=4) + 5*svd(train=train2, trueMat=trueMat, k=4))/10

    pred = []
    testusers = test['ncodpers'].tolist()

    n = 7
    for user in testusers[:]:
        j = u[user]
        p = max_items(UsV, UsV[j,:].argsort()[-n:][::-1], j)
        pred.append(" ".join(p))
        #print(len(p))
    test['added_products'] = pred
    test.to_csv('sub.csv', index=False)

In [9]:
recommend(test)

[[-8.37401350363357e-05 -1.7177463597197068e-05 -0.6029450761337395 ...,
  -0.05186413055731207 -0.056737162261541914 -0.12136951622894553]
 [-8.37401350363357e-05 -1.7177463597197068e-05 0.3970549238662605 ...,
  -0.05186413055731207 -0.056737162261541914 -0.12136951622894553]
 [-8.37401350363357e-05 -1.7177463597197068e-05 0.3970549238662605 ...,
  -0.05186413055731207 -0.056737162261541914 -0.12136951622894553]
 ..., 
 [-8.37401350363357e-05 -1.7177463597197068e-05 0.3970549238662605 ...,
  -0.05186413055731207 -0.056737162261541914 -0.12136951622894553]
 [-8.37401350363357e-05 -1.7177463597197068e-05 0.3970549238662605 ...,
  -0.05186413055731207 -0.056737162261541914 -0.12136951622894553]
 [-8.37401350363357e-05 -1.7177463597197068e-05 0.3970549238662605 ...,
  -0.05186413055731207 -0.056737162261541914 -0.12136951622894553]]
svd done
[[-8.309385396848692e-05 -1.7266255370075205e-05 -0.6062506002721594 ...,
  -0.04979911791017878 -0.054680072475106915 -0.120564865544432]
 [-8.3093

In [10]:
train2

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
657640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657788,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657795,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657790,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
657794,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657789,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
657787,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657777,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
657782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
trueMat

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])